# User Engagement Analysis

In [7]:
#importing libraries for this operation
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from statistics import *
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [8]:
# packages for path locations
import os
import sys
# package scripts in the script module folder 
sys.path.insert(0,'../scripts/')

In [9]:
df = pd.read_csv('../data/cleaned_telecom_data.csv')
df.head()

,MSISDN/Number,no.of xDR sessions,Total Uploads,Total Downloads,Total UL and DL,Youtube_Total_Data,Google_Total_Data,Email_Total_Data,Social_Media_Total_Data,Netflix_Total_Data,...,Avg Delay (ms),Dur. (ms),Dur. (ms).1,Activity_Duration(ms),Avg Throughput (kbps),Handset Manufacturer,Handset Type,Last Location Name,IMSI,IMEI
0,3.360100e+10,1,3.674974e+13,4.806241e+14,5.173738e+14,1.835594e+13,2.905912e+12,3.701304e+12,1.570185e+12,1.785519e+13,...,47.0,1823652.0,1.823653e+09,76411.0,67.0,Samsung,Samsung Galaxy A5 Sm-A520F,9164566995485190,2.082014e+14,3.552121e+13
1,3.360100e+10,1,5.380039e+13,1.180289e+15,1.234090e+15,3.935912e+13,4.414096e+12,9.373850e+11,1.933278e+12,3.556554e+13,...,70.0,1365104.0,1.365104e+09,3728.0,42.0,Samsung,Samsung Galaxy J5 (Sm-J530),L77566A,2.082019e+14,3.579401e+13
2,3.360100e+10,1,2.788364e+13,6.904999e+14,7.183836e+14,3.442524e+13,1.022912e+13,3.363124e+12,1.726277e+12,2.375120e+13,...,70.0,1361762.0,1.361763e+09,0.0,15.0,Samsung,Samsung Galaxy A8 (2018),D42335A,2.082003e+14,3.528151e+13
3,3.360101e+10,1,4.332422e+13,1.595068e+15,1.638393e+15,3.653476e+13,1.181176e+13,2.070983e+12,6.574930e+11,1.509259e+13,...,70.0,1321509.0,1.321510e+09,41212.0,88.0,undefined,undefined,T21824A,2.082014e+14,3.535661e+13
4,3.360101e+10,2,3.854281e+13,1.119848e+15,1.158391e+15,3.422225e+13,7.748843e+12,2.110349e+12,9.127880e+11,1.753980e+13,...,70.0,1089009.0,1.089009e+09,0.0,15.0,Samsung,Samsung Sm-G390F,D88865A,2.082014e+14,3.540701e+13


In the current dataset you’re expected to track the user’s engagement using the following engagement metrics:
* sessions frequency
* the duration of the session
* the sessions total traffic (download and upload (bytes))

In [15]:
df['sessions frequency'] = df.groupby('MSISDN/Number')['no.of xDR sessions'].transform('sum')

In [23]:
df_new = df[['MSISDN/Number', 'no.of xDR sessions', 'Total UL and DL', 'Dur. (ms)', 'sessions frequency']]

In [24]:
df_new

,MSISDN/Number,no.of xDR sessions,Total UL and DL,Dur. (ms),sessions frequency
0,3.360100e+10,1,5.173738e+14,1823652.0,1
1,3.360100e+10,1,1.234090e+15,1365104.0,1
2,3.360100e+10,1,7.183836e+14,1361762.0,1
3,3.360101e+10,1,1.638393e+15,1321509.0,1
4,3.360101e+10,2,1.158391e+15,1089009.0,2
...,...,...,...,...,...
106851,3.379000e+10,1,9.215483e+14,166996.0,1
106852,3.379000e+10,1,9.993291e+14,158070.0,1
106853,3.197021e+12,1,6.968673e+14,171428.0,1
106854,3.370000e+14,1,8.822031e+14,163575.0,1
